# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

import statsmodels.formula.api as stmf
import statsmodels.api as stm

import patsy

In [2]:
pd.options.display.float_format = '{:.3f}'.format

##### Leitura do dataframe

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

##### tratamento dos valores faltantes, trocar valores não preenchidos na idade pela mediana

In [4]:
df.fillna(value = df['idade'].median(), inplace = True )

###### remoção de colunas que não serão usadas

In [5]:
drop_colunas = ['Unnamed: 0', 'data_ref', 'id_cliente']
df = df.drop(drop_colunas, axis=1)

In [6]:
df.head(3)

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.603,1.000,8060.340
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.184,2.000,1852.150
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838,2.000,2253.890


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

### Abaixo, será ajustado um modelo para prever log(renda) em função de todas as outras variáveis. 
Nos resultados, pode-se notar que:

- **Sexo:** tem uma boa significancia no modelo.
- **posse_de_veiculo:** tem uma boa significancia no modelo.
- **posse_de_imovel:** tem uma boa significancia no modelo.
- **qtd_filhos:** tendo como base a qtd_filhos = 0, para as quantidades 1 e 2 temos uma diferença significante, acima disso não é estatisticamente relevante. 
- **tipo_renda:** com exceção de bolsista, todos os outros valores apresentam uma diferença significante com relação a "assalariado".
- **educacao:** a única diferença significante é de "secundário" para "superior completo".
- **estado_civil:** há uma diferença significante apenas entre casado/viúvo/solteiro.
- **tipo_residencia:** não há significancia alguma para o modelo.
- **idade:** é muito significante para o modelo.
- **tempo_emprego:** é muito significante para o modelo.
- **qt_pessoas_residencia:** tomando como base qt_pessoas_residencia=2, há uma diferença significante para 1, 3 e 4.

In [7]:
formula = "np.log(renda) ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(qtd_filhos, Treatment(0)) + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
C(tipo_residencia, Treatment('Casa')) + \
idade + \
tempo_emprego + \
C(qt_pessoas_residencia, Treatment(2.0))"


y, X = patsy.dmatrices(data = df,
                      formula_like = formula)


modelo = stm.OLS(y, X).fit()


modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     232.8
Date:                Mon, 24 Apr 2023   Prob (F-statistic):               0.00
Time:                        09:28:53   Log-Likelihood:                -16241.
No. Observations:               15000   AIC:                         3.255e+04
Df Residuals:                   14965   BIC:                         3.282e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1146      0.037    193.970      0.000       7.043       7.187
sexo[T.M]                                                       0.7951      0.014     57.504      0.000       0.768       0.822
posse_de_veiculo[T.True]                                        0.0349      0.013      2.666      0.008       0.009       0.061
posse_de_imovel[T.True]                                         0.0878      0.013      6.808      0.000       0.063       0.113
C(qtd_filhos, Treatment(0))[T.1]                               -0.2329      0.108     -2.148      0.032      -0.445      -0.020
C(qtd_filhos, Treatment(0))[T.2]                               -0.4387      0.216     -2.027      0.043      -0.863      -0.015
C(qtd_filhos, Treatment(0))[T.3]                               -0.3775      0.431     -0.877      0.381      -1.222       0.467
C(qtd_filhos, Treatment(0))[T.4]                                0.1967      0.425      0.463      0.643      -0.636       1.029
C(qtd_filhos, Treatment(0))[T.5]                                0.1522      0.407      0.374      0.708      -0.645       0.949
C(qtd_filhos, Treatment(0))[T.7]                                0.0204      0.160      0.128      0.899      -0.294       0.334
C(qtd_filhos, Treatment(0))[T.14]                              -0.4466      0.259     -1.726      0.084      -0.954       0.061
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2136      0.239      0.894      0.371      -0.255       0.682
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1520      0.015     10.281      0.000       0.123       0.181
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -2.3192      0.038    -61.533      0.000      -2.393      -2.245
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0578      0.022      2.624      0.009       0.015       0.101
C(educacao, Treatment('Secundário'))[T.Primário]                0.0693      0.056      1.228      0.219      -0.041       0.180
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1191      0.141      0.844      0.398      -0.157       0.396
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1222      0.013      9.610      0.000       0.097       0.147
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0309      0.031     -0.988      0.323      -0.092       0.030
C(estado_civil, Treatment('Casado'))[T.Separado]                0.2819      0.108      2.600      0.009       0.069       0.494
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2444      0.107      2.291      0.022       0.035       0.453
C(estado_ci

### Remoção da variável menos significante do modelo (tipo_residencia)
Nos resultados, pode-se notar que:

- **R-squared:** Não houve mudança.
- **R-squared ajustado:** Houve um aumento.
- **AIC:** Houve uma queda.

Tendo em vista os dados acima, tudo indica que a remoção da variável "tipo_residencia" melhorou o modelo. Visto que o R-squared não teve mudança, mas o R2 ajustado aumentou (quanto maior, melhor) e o AIC diminuiu (quanto menor, melhor).

In [8]:
formula = "np.log(renda) ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(qtd_filhos, Treatment(0)) + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
idade + \
tempo_emprego + \
C(qt_pessoas_residencia, Treatment(2.0))"


y, X = patsy.dmatrices(data = df,
                      formula_like = formula)


modelo = stm.OLS(y, X).fit()


modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     272.8
Date:                Mon, 24 Apr 2023   Prob (F-statistic):               0.00
Time:                        09:28:53   Log-Likelihood:                -16242.
No. Observations:               15000   AIC:                         3.254e+04
Df Residuals:                   14970   BIC:                         3.277e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1193      0.036    198.007      0.000       7.049       7.190
sexo[T.M]                                                       0.7961      0.014     57.743      0.000       0.769       0.823
posse_de_veiculo[T.True]                                        0.0347      0.013      2.657      0.008       0.009       0.060
posse_de_imovel[T.True]                                         0.0854      0.013      6.740      0.000       0.061       0.110
C(qtd_filhos, Treatment(0))[T.1]                               -0.2330      0.108     -2.149      0.032      -0.445      -0.020
C(qtd_filhos, Treatment(0))[T.2]                               -0.4407      0.216     -2.037      0.042      -0.865      -0.017
C(qtd_filhos, Treatment(0))[T.3]                               -0.3785      0.431     -0.879      0.379      -1.222       0.466
C(qtd_filhos, Treatment(0))[T.4]                                0.1745      0.424      0.411      0.681      -0.657       1.006
C(qtd_filhos, Treatment(0))[T.5]                                0.1300      0.406      0.320      0.749      -0.666       0.926
C(qtd_filhos, Treatment(0))[T.7]                                0.0188      0.160      0.117      0.907      -0.295       0.333
C(qtd_filhos, Treatment(0))[T.14]                              -0.4481      0.259     -1.732      0.083      -0.955       0.059
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2138      0.239      0.895      0.371      -0.255       0.682
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1525      0.015     10.331      0.000       0.124       0.181
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -2.3191      0.038    -61.584      0.000      -2.393      -2.245
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0586      0.022      2.665      0.008       0.016       0.102
C(educacao, Treatment('Secundário'))[T.Primário]                0.0667      0.056      1.185      0.236      -0.044       0.177
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1169      0.141      0.829      0.407      -0.160       0.393
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1215      0.013      9.572      0.000       0.097       0.146
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0315      0.031     -1.008      0.313      -0.093       0.030
C(estado_civil, Treatment('Casado'))[T.Separado]                0.2835      0.108      2.615      0.009       0.071       0.496
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2468      0.107      2.314      0.021       0.038       0.456
C(estado_ci

### Dropando tipo_renda = Bolsista, apenas 9 casos na base toda e estão piorando o modelo

In [9]:
df_tratado = df.copy()

#Removover linhas com "Bolsista"
df_tratado.drop(df_tratado.loc[ (df_tratado['tipo_renda'] == 'Bolsista') , : ].index, inplace = True)

In [10]:
formula = "np.log(renda) ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(qtd_filhos, Treatment(0)) + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
idade + \
tempo_emprego + \
qt_pessoas_residencia"


y, X = patsy.dmatrices(data = df_tratado,
                      formula_like = formula)


modelo = stm.OLS(y, X).fit()


modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     329.4
Date:                Mon, 24 Apr 2023   Prob (F-statistic):               0.00
Time:                        09:28:53   Log-Likelihood:                -16237.
No. Observations:               14991   AIC:                         3.252e+04
Df Residuals:                   14966   BIC:                         3.272e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5926      0.212     31.089      0.000       6.177       7.008
sexo[T.M]                                                       0.7958      0.014     57.758      0.000       0.769       0.823
posse_de_veiculo[T.True]                                        0.0345      0.013      2.639      0.008       0.009       0.060
posse_de_imovel[T.True]                                         0.0854      0.013      6.740      0.000       0.061       0.110
C(qtd_filhos, Treatment(0))[T.1]                               -0.2282      0.106     -2.162      0.031      -0.435      -0.021
C(qtd_filhos, Treatment(0))[T.2]                               -0.4514      0.210     -2.153      0.031      -0.862      -0.040
C(qtd_filhos, Treatment(0))[T.3]                               -0.8284      0.318     -2.604      0.009      -1.452      -0.205
C(qtd_filhos, Treatment(0))[T.4]                               -0.6877      0.453     -1.519      0.129      -1.575       0.200
C(qtd_filhos, Treatment(0))[T.5]                               -0.9034      0.727     -1.242      0.214      -2.329       0.522
C(qtd_filhos, Treatment(0))[T.7]                               -1.8091      0.799     -2.265      0.024      -3.374      -0.244
C(qtd_filhos, Treatment(0))[T.14]                              -4.3225      1.548     -2.792      0.005      -7.358      -1.287
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1527      0.015     10.342      0.000       0.124       0.182
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -2.3190      0.038    -61.641      0.000      -2.393      -2.245
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0585      0.022      2.659      0.008       0.015       0.102
C(educacao, Treatment('Secundário'))[T.Primário]                0.0666      0.056      1.182      0.237      -0.044       0.177
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1167      0.141      0.828      0.408      -0.160       0.393
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1213      0.013      9.555      0.000       0.096       0.146
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0320      0.031     -1.024      0.306      -0.093       0.029
C(estado_civil, Treatment('Casado'))[T.Separado]                0.2811      0.108      2.614      0.009       0.070       0.492
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2442      0.105      2.316      0.021       0.038       0.451
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0381      0.023     -1.652      0.099      -0.083       0.007
C(estado_ci

##### Remover "educacao" do modelo, pois o p-valor só de uma categoria é significante

In [11]:
formula = "np.log(renda) ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(qtd_filhos, Treatment(0)) + \
C(tipo_renda, Treatment('Assalariado')) + \
C(estado_civil, Treatment('Casado')) + \
idade + \
tempo_emprego + \
qt_pessoas_residencia"


y, X = patsy.dmatrices(data = df_tratado,
                      formula_like = formula)


modelo = stm.OLS(y, X).fit()


modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     387.9
Date:                Mon, 24 Apr 2023   Prob (F-statistic):               0.00
Time:                        09:28:54   Log-Likelihood:                -16287.
No. Observations:               14991   AIC:                         3.262e+04
Df Residuals:                   14970   BIC:                         3.278e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6640      0.212     31.364      0.000       6.248       7.080
sexo[T.M]                                                       0.7885      0.014     57.140      0.000       0.761       0.816
posse_de_veiculo[T.True]                                        0.0444      0.013      3.398      0.001       0.019       0.070
posse_de_imovel[T.True]                                         0.0883      0.013      6.954      0.000       0.063       0.113
C(qtd_filhos, Treatment(0))[T.1]                               -0.2157      0.106     -2.037      0.042      -0.423      -0.008
C(qtd_filhos, Treatment(0))[T.2]                               -0.4234      0.210     -2.013      0.044      -0.836      -0.011
C(qtd_filhos, Treatment(0))[T.3]                               -0.8056      0.319     -2.525      0.012      -1.431      -0.180
C(qtd_filhos, Treatment(0))[T.4]                               -0.6848      0.454     -1.508      0.132      -1.575       0.206
C(qtd_filhos, Treatment(0))[T.5]                               -0.8962      0.730     -1.228      0.219      -2.327       0.534
C(qtd_filhos, Treatment(0))[T.7]                               -1.7736      0.801     -2.214      0.027      -3.344      -0.203
C(qtd_filhos, Treatment(0))[T.14]                              -4.2230      1.553     -2.719      0.007      -7.268      -1.178
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1634      0.015     11.075      0.000       0.134       0.192
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -2.3176      0.038    -61.448      0.000      -2.392      -2.244
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0784      0.022      3.569      0.000       0.035       0.121
C(estado_civil, Treatment('Casado'))[T.Separado]                0.2775      0.108      2.573      0.010       0.066       0.489
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2389      0.106      2.259      0.024       0.032       0.446
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0420      0.023     -1.818      0.069      -0.087       0.003
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.2706      0.109      2.486      0.013       0.057       0.484
idade                                                           0.0048      0.001      6.307      0.000       0.003       0.006
tempo_emprego                                                   0.0606      0.001     59.497      0.000       0.059       0.063
qt_pessoas_residencia                                           0.2535      0.105      2.418      0.016       0.048       0.459
===========

##### Remover 'qtd_filhos' e 'estado_civil' do modelo

In [13]:
formula = "np.log(renda) ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(tipo_renda, Treatment('Assalariado')) + \
idade + \
tempo_emprego + \
qt_pessoas_residencia"


y, X = patsy.dmatrices(data = df_tratado,
                      formula_like = formula)


modelo = stm.OLS(y, X).fit()


pd.options.display.float_format = '{:.3f}'.format
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     857.9
Date:                Mon, 24 Apr 2023   Prob (F-statistic):               0.00
Time:                        09:29:21   Log-Likelihood:                -16301.
No. Observations:               14991   AIC:                         3.262e+04
Df Residuals:                   14981   BIC:                         3.270e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1478      0.038    188.380      0.000       7.073       7.222
sexo[T.M]                                                       0.7862      0.014     57.233      0.000       0.759       0.813
posse_de_veiculo[T.True]                                        0.0449      0.013      3.452      0.001       0.019       0.070
posse_de_imovel[T.True]                                         0.0884      0.013      6.969      0.000       0.064       0.113
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1615      0.015     10.956      0.000       0.133       0.190
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -2.3197      0.038    -61.656      0.000      -2.393      -2.246
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0808      0.022      3.682      0.000       0.038       0.124
idade                                                           0.0046      0.001      6.354      0.000       0.003       0.006
tempo_emprego                                                   0.0607      0.001     59.587      0.000       0.059       0.063
qt_pessoas_residencia                                           0.0192      0.007      2.774      0.006       0.006       0.033
==============================================================================
Omnibus:                        3.373   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.185   Jarque-Bera (JB):                3.412
Skew:                           0.021   Prob(JB):                        0.182
Kurtosis:                       3.061   Cond. No.                         335.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Observações:

Após deixar o modelo apenas com as variávies com p-valor < 5%, o r-squared teve uma muito pequena, de menos de 1%.

Também houve uma queda no BIC, que quanto menor melhor, de 32.820 para 32.700.

Portanto, entende-se que o modelo final é melhor que o inicial.